# F-scale-train: This Notebook trains the NOIRE-Net F-region scaling networks

## 1 - Import libaries 

In [1]:
import os
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.metrics import mean_squared_error
from math import sqrt
from tensorflow.keras.callbacks import ModelCheckpoint
import pickle

## 2 - Define function to get F-region scaling parameters from .par file (item1 = fF, item2 = hF)

In [2]:
# The function get_regression_label_from_par reads a .par file and returns 
# the F-region maximum O-mode frequency and the F-region height.
# Both the fF and hF must be values (can not be 'nan'). 

def get_regression_label_from_par(par_file_path):
    try:
        # Open the file at the specified path
        with open(par_file_path, 'r') as file:
            content = file.readline().strip()  # Read the first line and remove leading/trailing whitespace
            items = content.split()  # Split the line into individual items

            # Check if both the first and third items are not 'nan' (not a number)
            # If they are both valid numbers, convert them to floats and return them as a tuple
            if items[0].lower() != 'nan' and items[2].lower() != 'nan':
                return float(items[0]), float(items[2])
            else:
                # If either item is 'nan', return None
                return None
    except Exception as e:
        # Print an error message if an exception occurs while processing the file
        print(f"Error reading {par_file_path}: {e}")
        # Return None if there is an error
        return None


## 3 - Define function to load ionograms and preprocess the data

In [3]:
# The load_data function loads and preprocesses image data from a specified directory,
# converting images to grayscale and resizing them, while also extracting corresponding
# scaling parameters from associated .par files for a regression task.

def load_data(data_dir, target_size=(310, 310)):
    images = []  # List to store preprocessed images
    labels = []  # List to store corresponding regression labels

    # Construct paths to the directories containing ionograms and parameters
    ionograms_dir = os.path.join(data_dir, 'ionograms')
    parameters_dir = os.path.join(data_dir, 'parameters')

    # Iterate over the files in the ionograms directory
    for filename in os.listdir(ionograms_dir):
        if filename.endswith('.png'):  # Check if the file is a PNG image
            # Construct full paths to the image file and its corresponding .par file
            img_path = os.path.join(ionograms_dir, filename)
            par_path = os.path.join(parameters_dir, filename.replace('.png', '.par'))

            # Load the image, convert it to grayscale, resize it, and normalize pixel values
            image = load_img(img_path, color_mode='grayscale', target_size=target_size)
            image = img_to_array(image) / 255.0  # Normalize image pixels to be between 0 and 1

            # Get the regression labels from the .par file
            regression_label = get_regression_label_from_par(par_path)
            
            # Proceed only if valid regression labels are found
            if regression_label is not None:
                images.append(image)
                labels.append(regression_label)

    # Convert the lists of images and labels to numpy arrays and return them
    return np.array(images), np.array(labels)

## 4 - Load the ionograms and scaling outputs from the data folder 

In [4]:
# Specify the directory where the data is stored
data_dir = 'train-val-test'  # 'train_test_val' should be replaced with the actual path to your data directory

# Call the load_data function to load and preprocess the data
# X contain the preprocessed images, and y will contain the corresponding labels
X, y = load_data(data_dir)

## 5 - Define a function to create the NOIRE-Net architecture

In [5]:
# This code defines and complies NOIRE-Net a convolutional neural network (CNN) model using Keras, 
# with multiple convolutional layers, batch normalization, max pooling, and dense layers, 
# designed for binary classification tasks.

def NOIREnet():
    model = Sequential([
    # First convolutional layer with 32 filters and a kernel size of 3x3
    # 'padding=same' ensures the output size is the same as the input size
    # 'input_shape' is set for the first layer to indicate the shape of the input data
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(310, 310, 1)),
    
    # Batch normalization to normalize the activations from the previous layer
    BatchNormalization(),

    # Second convolutional layer with 32 filters and a kernel size of 3x3
    Conv2D(32, (3, 3), activation='relu'),

    # Another batch normalization
    BatchNormalization(),

    # First max pooling layer to reduce spatial dimensions
    MaxPooling2D((2, 2)),

    # Repeating the pattern of two convolutional layers followed by batch normalization
    # and a max pooling layer, gradually increasing the number of filters
    Conv2D(32, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Flatten the output from the convolutional layers to feed into dense layers
    Flatten(),

    # Dense (fully connected) layer with 256 neurons and relu activation
    Dense(256, activation='relu'),

    # Dropout layer to reduce overfitting
    Dropout(0.5),

    # Another dense layer with 128 neurons
    Dense(128, activation='relu'),

    # Output layer with a single neuron and sigmoid activation for binary classification
    Dense(2, activation='linear')
    ])
    
    # Compile the CNN model
    model.compile(
        optimizer='adam',  # Using the Adam optimizer for adaptive learning rate optimization
        loss='mse',  # Mean squared error loss function, suitable for regression tasks
        metrics=['mse']  # The model will report 'mse' as a performance metric
    )
    
    # Return the compiled model
    return model

## 6 - Train 10 CNNs for F-region scaling and save the models

In [ ]:
# This code trains 10 Convolutional Neural Networks (CNNs) on differently split subsets
# of a dataset for binary classification,saves the best model of each training session, 
# and records their training histories.

import os
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Create the directory for saving models and histories if it doesn't exist
save_dir = 'F-scale'
os.makedirs(save_dir, exist_ok=True)

# Initialize lists to store the training histories and filenames of the best models
histories = []
model_filenames = []

# Define the ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,      # Factor to reduce the learning rate
    patience=20,     # Number of epochs with no improvement to wait before reducing LR
    min_lr=0.00001   # Minimum learning rate
)

# Loop to train 10 CNN models with different data splits
for i in range(10):
    # Split the dataset into training, validation and testing sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=i)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=i)

    # Create a new CNN model for each iteration
    model = NOIREnet()

    # Define the filename for the checkpoint model
    model_filename = os.path.join(save_dir, f'F-scale_run{i+1}.h5')

    # Define a checkpoint callback to save the best model based on validation accuracy
    checkpoint_callback = ModelCheckpoint(
        model_filename,
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        mode='min',
        save_weights_only=False
    )

    # Train the model with specified callbacks including ReduceLROnPlateau
    history = model.fit(
        X_train, y_train,
        batch_size=64,
        epochs=100,
        validation_data=(X_val, y_val),
        callbacks=[checkpoint_callback, reduce_lr]  # Include ReduceLROnPlateau callback
    )

    # Save the training history and the filename of the saved best model
    histories.append(history.history)
    model_filenames.append(model_filename)

# Optionally, save the training histories to a file in the same 'E-scale' directory
history_filename = os.path.join(save_dir, 'training_histories.pkl')
with open(history_filename, 'wb') as file:
    pickle.dump({'histories': histories, 'model_filenames': model_filenames}, file)


2023-12-02 17:40:42.669508: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-02 17:40:42.669638: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Max
Epoch 1/100


2023-12-02 17:40:44.132788: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-12-02 17:40:45.085621: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


83/83 [==============================] - ETA: 0s - loss: 1162.2473 - mse: 1162.2473

2023-12-02 17:41:56.796952: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 1895.93127, saving model to F-scale/F-scale_run1.h5
83/83 [==============================] - 79s 877ms/step - loss: 1162.2473 - mse: 1162.2473 - val_loss: 1895.9313 - val_mse: 1895.9313 - lr: 0.0010
Epoch 2/100
83/83 [==============================] - ETA: 0s - loss: 113.0806 - mse: 113.0806
Epoch 2: val_loss did not improve from 1895.93127
83/83 [==============================] - 58s 693ms/step - loss: 113.0806 - mse: 113.0806 - val_loss: 1898.8041 - val_mse: 1898.8041 - lr: 0.0010
Epoch 3/100
83/83 [==============================] - ETA: 0s - loss: 67.8424 - mse: 67.8424
Epoch 3: val_loss improved from 1895.93127 to 1057.65454, saving model to F-scale/F-scale_run1.h5
83/83 [==============================] - 69s 832ms/step - loss: 67.8424 - mse: 67.8424 - val_loss: 1057.6545 - val_mse: 1057.6545 - lr: 0.0010
Epoch 4/100
83/83 [==============================] - ETA: 0s - loss: 55.4795 - mse: 55.4795
Epoch 4: val_loss did not improve from 1057.654